In [1]:
# Cell 1: Imports and I/O paths

import pandas as pd
import numpy as np
from pathlib import Path

# I/O
EXCEL_PATH = Path("data/raw/DataSheet_ETH_250902.xlsx")  # provided file
OUTPUT_CSV = Path("data/processed/eth_spray_merged_preprocessed.csv")

# Pandas options
pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 80)

In [2]:
# Cell 2: Helper functions

def read_multiheader(sheet_name):
    """
    Read a sheet that uses a two-row header (top: variable block,
    second: ETH-xx run identifiers), returning a DataFrame with a
    MultiIndex for columns.
    """
    df = pd.read_excel(EXCEL_PATH, sheet_name=sheet_name, header=[0, 1])
    # Normalize column level names
    df.columns = pd.MultiIndex.from_tuples(
        [(str(t[0]).strip(), (str(t[1]).strip() if not pd.isna(t[1]) else "")) for t in df.columns],
        names=["var", "run"]
    )
    return df

def get_time_series(df_multi):
    """
    Extract the Time(ms) series from a two-level column DataFrame.
    """
    time_cols = [c for c in df_multi.columns if c[0] == "Time(ms)"]
    assert len(time_cols) == 1, "Expected exactly one Time(ms) column"
    return df_multi[time_cols[0]].astype(float).rename("Time_ms")

def stack_block(df_multi, top_name, value_name):
    """
    For a given top-level column block (e.g., 'Chamber pressure (bar)'),
    produce a long DataFrame with ['run', 'Time_ms', value_name].
    """
    time = get_time_series(df_multi)
    # Sub-block for the variable across ETH runs
    block_cols = [c for c in df_multi.columns if c[0] == top_name and c[1] != ""]
    sub = df_multi[block_cols].copy()
    # Rename columns from ('var', 'ETH-xx') -> 'ETH-xx'
    sub.columns = [c[1] for c in block_cols]
    sub.insert(0, "Time_ms", time.values)
    long = sub.melt(id_vars="Time_ms", var_name="run", value_name=value_name)
    # Ensure numeric
    long[value_name] = pd.to_numeric(long[value_name], errors="coerce")
    long["Time_ms"] = pd.to_numeric(long["Time_ms"], errors="coerce")
    return long

def groupwise_ffill_bfill(df, group_cols, order_cols, cols_to_fill=None):
    """
    Within each group, sort and apply ffill followed by bfill for selected columns.
    If cols_to_fill is None, all non-key numeric columns are filled.
    """
    df = df.sort_values(group_cols + order_cols).copy()
    if cols_to_fill is None:
        keyset = set(group_cols + order_cols)
        cols_to_fill = [c for c in df.columns if c not in keyset and pd.api.types.is_numeric_dtype(df[c])]
    def _fill(g):
        g[cols_to_fill] = g[cols_to_fill].ffill().bfill()
        return g
    return df.groupby(group_cols, as_index=False, group_keys=False).apply(_fill)


In [3]:
# Cell 3: Inputs from "Exp. Conditions in Time"

ect = read_multiheader("Exp. Conditions in Time")

inp_pc   = stack_block(ect, "Chamber pressure (bar)",   "Pc_bar")
inp_tc   = stack_block(ect, "Chamber temperature (K)",  "Tc_K")
inp_pinj = stack_block(ect, "Injection pressure (bar)", "Pinj_bar")
inp_rho  = stack_block(ect, "Density (kg/m3)",          "rho_kgm3")
inp_mu   = stack_block(ect, "Viscosity (Pas)",          "mu_Pas")

# Merge all input blocks on ['run','Time_ms'] using outer joins to preserve full grids
from functools import reduce
inputs = reduce(
    lambda left, right: pd.merge(left, right, on=["run", "Time_ms"], how="outer"),
    [inp_pc, inp_tc, inp_pinj, inp_rho, inp_mu]
)

# Coerce dtypes
for col in ["Pc_bar", "Tc_K", "Pinj_bar", "rho_kgm3", "mu_Pas", "Time_ms"]:
    inputs[col] = pd.to_numeric(inputs[col], errors="coerce")

# Optional: ensure canonical ETH run ordering
inputs["run"] = inputs["run"].astype(str)
inputs.head()


,Time_ms,run,Pc_bar,Tc_K,Pinj_bar,rho_kgm3,mu_Pas
0,0.000,ETH-01,55.03180,192.029519,98.864550,810.720228,0.001879
1,0.025,ETH-01,55.00570,192.015831,98.874062,810.718262,0.001879
2,0.050,ETH-01,55.00810,191.988228,98.907356,810.718443,0.001879
3,0.075,ETH-01,55.01635,192.081988,98.855037,810.719065,0.001879
4,0.100,ETH-01,55.01250,191.988000,98.878819,810.718775,0.001879


In [4]:
# Cell 4: Targets from angle/penetration sheets

# Shadow: angle (smoothed)
shadow_angle_df = read_multiheader("Spray Angle (Shadow)")
tgt_ang_sh = stack_block(shadow_angle_df, "Smoothed angle (deg)", "angle_shadow_deg")

# Shadow: penetration (L/D)
shadow_len_df = read_multiheader("Spray Penetration (Shadow)")
tgt_len_sh = stack_block(shadow_len_df, "Penetration (L/D)", "len_shadow_L_D")

# Mie: angle (smoothed)
mie_angle_df = read_multiheader("Spray Angle (Mie)")
tgt_ang_mie = stack_block(mie_angle_df, "Smoothed angle (deg)", "angle_mie_deg")

# Mie: penetration (L/D)
mie_len_df = read_multiheader("Spray Penetration (Mie)")
tgt_len_mie = stack_block(mie_len_df, "Penetration (L/D)", "len_mie_L_D")

# Coerce numerics
for df_ in [tgt_ang_sh, tgt_len_sh, tgt_ang_mie, tgt_len_mie]:
    for c in df_.columns:
        if c not in ["run"]:
            df_[c] = pd.to_numeric(df_[c], errors="coerce")

# Quick check
tgt_ang_sh.head()


,Time_ms,run,angle_shadow_deg
0,0.000,ETH-01,NaN
1,0.025,ETH-01,NaN
2,0.050,ETH-01,NaN
3,0.075,ETH-01,NaN
4,0.100,ETH-01,NaN


In [5]:
# Cell 5: Merge all pieces

# Progressive outer merges to preserve all time points per run
merged = inputs.merge(tgt_ang_sh, on=["run", "Time_ms"], how="outer")
merged = merged.merge(tgt_len_sh,  on=["run", "Time_ms"], how="outer")
merged = merged.merge(tgt_ang_mie, on=["run", "Time_ms"], how="outer")
merged = merged.merge(tgt_len_mie, on=["run", "Time_ms"], how="outer")

# Standardize column order
cols_order = [
    "run", "Time_ms",
    "Pc_bar", "Tc_K", "Pinj_bar", "rho_kgm3", "mu_Pas",
    "angle_shadow_deg", "len_shadow_L_D",
    "angle_mie_deg", "len_mie_L_D"
]
merged = merged[cols_order]

# Coerce numerics again
for c in merged.columns:
    if c not in ["run"]:
        merged[c] = pd.to_numeric(merged[c], errors="coerce")

merged.head()


,run,Time_ms,Pc_bar,Tc_K,Pinj_bar,rho_kgm3,mu_Pas,angle_shadow_deg,len_shadow_L_D,angle_mie_deg,len_mie_L_D
0,ETH-01,0.000,55.03180,192.029519,98.864550,810.720228,0.001879,NaN,NaN,NaN,NaN
1,ETH-01,0.025,55.00570,192.015831,98.874062,810.718262,0.001879,NaN,NaN,NaN,NaN
2,ETH-01,0.050,55.00810,191.988228,98.907356,810.718443,0.001879,NaN,NaN,NaN,NaN
3,ETH-01,0.075,55.01635,192.081988,98.855037,810.719065,0.001879,NaN,13.126559,NaN,17.571262
4,ETH-01,0.100,55.01250,191.988000,98.878819,810.718775,0.001879,NaN,20.204667,NaN,24.506565


In [6]:
# Check the shape of merged data
print("Shape of merged data:", merged.shape)
print("Number of unique runs:", merged['run'].nunique())
print("Unique runs:", sorted(merged['run'].unique()))
print("Time points per run (first few runs):")
for run in sorted(merged['run'].unique())[:3]:
    count = merged[merged['run'] == run].shape[0]
    print(f"  {run}: {count} time points")

Shape of merged data: (847, 11)
Number of unique runs: 7
Unique runs: ['ETH-01', 'ETH-02', 'ETH-03', 'ETH-04', 'ETH-05', 'ETH-06', 'ETH-06.1']
Time points per run (first few runs):
  ETH-01: 121 time points
  ETH-02: 121 time points
  ETH-03: 121 time points


In [7]:
# Cell 6: Groupwise ffill and bfill within each run

key_cols = ["run", "Time_ms"]
num_cols = [c for c in merged.columns if c not in key_cols]

filled = groupwise_ffill_bfill(
    merged.copy(),
    group_cols=["run"],
    order_cols=["Time_ms"],
    cols_to_fill=num_cols
)

# Optional sanity: if any rows still have NaNs, they would be rare; inspect counts
na_summary = filled.isna().sum()
na_summary


C:\temp\ipykernel_24308\2757019269.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby(group_cols, as_index=False, group_keys=False).apply(_fill)


run                   0
Time_ms               0
Pc_bar              121
Tc_K                121
Pinj_bar            121
rho_kgm3            121
mu_Pas              121
angle_shadow_deg    121
len_shadow_L_D      121
angle_mie_deg       121
len_mie_L_D         121
dtype: int64

In [11]:
# Cell 7: Finalize and write CSV

# Sort deterministically
filled = filled.sort_values(["run", "Time_ms"]).reset_index(drop=True)

# Integrity checks
assert filled[["run", "Time_ms"]].drop_duplicates().shape[0] == filled.shape[0], "Duplicate (run, Time_ms) rows detected"
expected_cols = {
    "run", "Time_ms",
    "Pc_bar", "Tc_K", "Pinj_bar", "rho_kgm3", "mu_Pas",
    "angle_shadow_deg", "len_shadow_L_D",
    "angle_mie_deg", "len_mie_L_D"
}
assert expected_cols.issubset(set(filled.columns)), "Missing expected columns"

# Save
filled.to_csv(OUTPUT_CSV, index=False)

# Brief report
print("Wrote:", OUTPUT_CSV.resolve())
print("Shape:", filled.shape)
print("Columns:", list(filled.columns))
print("Remaining NaNs per column:\n", filled.isna().sum())

Wrote: D:\Yuvin Raja\GitHub Repos\spray-vision\data\processed\eth_spray_merged_preprocessed.csv
Shape: (847, 11)
Columns: ['run', 'Time_ms', 'Pc_bar', 'Tc_K', 'Pinj_bar', 'rho_kgm3', 'mu_Pas', 'angle_shadow_deg', 'len_shadow_L_D', 'angle_mie_deg', 'len_mie_L_D']
Remaining NaNs per column:
 run                   0
Time_ms               0
Pc_bar              121
Tc_K                121
Pinj_bar            121
rho_kgm3            121
mu_Pas              121
angle_shadow_deg    121
len_shadow_L_D      121
angle_mie_deg       121
len_mie_L_D         121
dtype: int64


In [9]:
# Let's investigate the discrepancy in detail
print("Expected: 726 rows (66 time points × 11 runs)")
print("But we have 7 runs, not 11:")
print("Runs found:", sorted(filled['run'].unique()))
print("\nTime points per run:")
for run in sorted(filled['run'].unique()):
    count = filled[filled['run'] == run].shape[0]
    print(f"  {run}: {count} time points")

# Let's check what 726 should be
print(f"\nActual total: {filled.shape[0]} rows")
print(f"Expected: 726 rows")
print(f"Difference: {filled.shape[0] - 726}")

# Check if there are duplicate runs or time points
print(f"\nChecking for duplicates:")
duplicates = filled.groupby(['run', 'Time_ms']).size()
duplicates_found = duplicates[duplicates > 1]
if len(duplicates_found) > 0:
    print("Found duplicate (run, Time_ms) pairs:")
    print(duplicates_found.head())
else:
    print("No duplicate (run, Time_ms) pairs found")

# Check the time range for each run
print(f"\nTime range per run:")
for run in sorted(filled['run'].unique()):
    run_data = filled[filled['run'] == run]
    min_time = run_data['Time_ms'].min()
    max_time = run_data['Time_ms'].max()
    print(f"  {run}: {min_time:.3f} to {max_time:.3f} ms")

Expected: 726 rows (66 time points × 11 runs)
But we have 7 runs, not 11:
Runs found: ['ETH-01', 'ETH-02', 'ETH-03', 'ETH-04', 'ETH-05', 'ETH-06', 'ETH-06.1']

Time points per run:
  ETH-01: 121 time points
  ETH-02: 121 time points
  ETH-03: 121 time points
  ETH-04: 121 time points
  ETH-05: 121 time points
  ETH-06: 121 time points
  ETH-06.1: 121 time points

Actual total: 847 rows
Expected: 726 rows
Difference: 121

Checking for duplicates:
No duplicate (run, Time_ms) pairs found

Time range per run:
  ETH-01: 0.000 to 3.000 ms
  ETH-02: 0.000 to 3.000 ms
  ETH-03: 0.000 to 3.000 ms
  ETH-04: 0.000 to 3.000 ms
  ETH-05: 0.000 to 3.000 ms
  ETH-06: 0.000 to 3.000 ms
  ETH-06.1: 0.000 to 3.000 ms


In [10]:
# Let's check the original Excel file to see how many runs were expected
import pandas as pd

# Read one of the sheets to see the actual structure
try:
    sheet_df = pd.read_excel(EXCEL_PATH, sheet_name="Exp. Conditions in Time", header=[0, 1])
    print("Columns in the Excel file:")
    for i, col in enumerate(sheet_df.columns):
        print(f"  {i}: {col}")
    
    # Count the ETH runs in the columns
    eth_runs = [col[1] for col in sheet_df.columns if col[1].startswith('ETH') and col[1] != '']
    print(f"\nETH runs found in Excel: {len(eth_runs)}")
    print("Runs:", sorted(set(eth_runs)))
    
except Exception as e:
    print(f"Error reading Excel: {e}")

# Calculate what 726 would mean
print(f"\n726 rows analysis:")
print(f"726 ÷ 66 time points = {726/66:.1f} runs")
print(f"726 ÷ 121 time points = {726/121:.1f} runs")
print(f"Our actual: 847 ÷ 121 = {847/121:.1f} runs")

# Maybe the expectation was wrong?
print(f"\nPossible explanations:")
print(f"1. Expected 66 time points per run, but actually have 121")
print(f"2. Expected 11 runs, but actually have 7 (including ETH-06.1)")
print(f"3. The 726 expectation might be incorrect")

Columns in the Excel file:
  0: ('Time(ms)', 'Unnamed: 0_level_1')
  1: ('Chamber pressure (bar)', 'ETH-01')
  2: ('Chamber pressure (bar)', 'ETH-02')
  3: ('Chamber pressure (bar)', 'ETH-03')
  4: ('Chamber pressure (bar)', 'ETH-04')
  5: ('Chamber pressure (bar)', 'ETH-05')
  6: ('Chamber pressure (bar)', 'ETH-06')
  7: ('Chamber pressure (bar)', 'ETH-06.1')
  8: ('Chamber temperature (K)', 'ETH-01')
  9: ('Chamber temperature (K)', 'ETH-02')
  10: ('Chamber temperature (K)', 'ETH-03')
  11: ('Chamber temperature (K)', 'ETH-04')
  12: ('Chamber temperature (K)', 'ETH-05')
  13: ('Chamber temperature (K)', 'ETH-06')
  14: ('Chamber temperature (K)', 'ETH-06.1')
  15: ('Injection pressure (bar)', 'ETH-01')
  16: ('Injection pressure (bar)', 'ETH-02')
  17: ('Injection pressure (bar)', 'ETH-03')
  18: ('Injection pressure (bar)', 'ETH-04')
  19: ('Injection pressure (bar)', 'ETH-05')
  20: ('Injection pressure (bar)', 'ETH-06')
  21: ('Injection pressure (bar)', 'ETH-06.1')
  22: ('Dens